In [1]:
%load_ext autoreload
%autoreload 2

# Добавляем путь к основной папке проекта, чтобы иметь возможность делать импорт из src
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pathlib import Path

import pandas as pd
import catboost

In [8]:
additional_data_path = Path('../classifier_retraining')

block_files = [p for p in sorted(additional_data_path.iterdir()) if p.suffix == '.csv']

In [12]:
block_data_chunks = []

for p in block_files:
    data = pd.read_csv(p)
    block_data_chunks.append(data)
all_data = pd.concat(block_data_chunks)
all_data.shape

(9069895, 25)

In [15]:
all_data.columns

Index(['hash', 'from', 'nonce', 'maxFeePerGas', 'maxPriorityFeePerGas', 'gas',
       'already_waiting', 'included_into_next_block', 'baseFeePerGas',
       'block_number', 'availablePriorityFee', 'mempool', 'prev_block_gasUsed',
       'prev_block_baseFeePerGas', 'change_baseFeePerGas',
       'total_eligible_txs', 'cumulative_gas', 'first_gas_unit',
       'last_gas_unit', 'priority_n', 'priority_percent', 'timestamp', 'hour',
       'prediction', 'status'],
      dtype='object')

In [26]:
compliant = all_data[all_data['status'] == 1]
print(compliant['hash'].nunique())
compliant.shape

7298000


(7298000, 25)

In [27]:
non_compliant = all_data[all_data['status'] == -1]
print(non_compliant['hash'].nunique())
non_compliant.shape

1010


(1010, 25)

In [28]:
unknown_at_block_time = all_data[all_data['status'] == 0]
print(unknown_at_block_time['hash'].nunique())
unknown_at_block_time.shape

843172


(1770885, 25)

In [ ]:
## Заполним статусы для для тех, по которым не было информации на момент блока

In [32]:
all_data['compliance_status'] = 0
all_data.loc[all_data['hash'].isin(compliant['hash'].unique()), 'compliance_status'] = 1
all_data.loc[all_data['hash'].isin(non_compliant['hash'].unique()), 'compliance_status'] = -1

In [33]:
all_data['compliance_status'].value_counts()

 1    8710792
 0     356473
-1       2630
Name: compliance_status, dtype: int64

In [34]:
all_data[all_data['compliance_status'] == 0]

,hash,from,nonce,maxFeePerGas,maxPriorityFeePerGas,gas,already_waiting,included_into_next_block,baseFeePerGas,block_number,...,cumulative_gas,first_gas_unit,last_gas_unit,priority_n,priority_percent,timestamp,hour,prediction,status,compliance_status
3,0xb78315768262de8edc44893999e22c0577d1953a6648...,0x4E4B37D0a102D7e7213155A979a5b36A46ad3da5,85,37.671893,37.671893,285088,2215,False,22.718593,16767465,...,912234,0.020905,0.030408,3,0.025210,1678083227,6,0,0,0
5,0x8597fffd77f42e8a8ef1bf638ed23b7e80e5f5023a61...,0x58803DB3cC22e8b1562C332494DA49CACD94C6aB,61,32.605728,32.605728,21000,74,False,22.718593,16767465,...,979862,0.031962,0.032662,5,0.042017,1678083227,6,1,0,0
10,0xb9407f18e14a2cdff1489eefa027724363e1c03485da...,0xA873bBA4d7f39Be4960b54A534910863Fa66d884,7,29.000000,29.000000,21000,2572,False,22.718593,16767465,...,1224574,0.040119,0.040819,10,0.084034,1678083227,6,0,0,0
11,0xcdfe8302218331a7f98d9107c26634cc908e9ab87944...,0xD21BeDab06404A1d5500Fb645254134391740e8B,6,29.000000,29.000000,21000,2572,False,22.718593,16767465,...,1245574,0.040819,0.041519,11,0.092437,1678083227,6,0,0,0
13,0xd314489e5ce32bcf2ab74242b4fac0452f36a454806f...,0xFD98ec94860F574738596d9c843eb264574154A6,10,28.445986,28.445986,21680,1125,False,22.718593,16767465,...,1288254,0.042219,0.042942,13,0.109244,1678083227,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,0x1f44a030a8a31a577f90f3a7e7572054a6e4806ef5de...,0xfcc27B999c8c37FE5a220BC5817CFf95f7C51494,1586,25.292847,0.100000,46593,0,False,17.445109,16817663,...,26793135,0.891551,0.893104,290,0.845481,1678693451,7,0,0,0
292,0x1dab16aa0a7d94b3a76bb6ddea329bca1759f807a445...,0xD72291eF964a078d2F18D7C462739Cd40d496D02,15,24.878188,0.100000,370000,1481,False,17.445109,16817663,...,27226332,0.895211,0.907544,292,0.851312,1678693451,7,0,0,0
309,0xcd6beadadc0e9f28ad27956925292a6257a40439ca62...,0x9CceD06933250ce1a11a1aF64f710CF7cE97c822,2,25.292847,0.100000,92046,0,False,17.445109,16817663,...,28765874,0.955794,0.958862,309,0.900875,1678693451,7,0,0,0
317,0x01eda80e634809f103a93d7b94e010bd7434bb61c597...,0xD72291eF964a078d2F18D7C462739Cd40d496D02,13,21.675961,0.100000,370000,1481,False,17.445109,16817663,...,30095207,0.990840,1.003174,317,0.924198,1678693451,7,0,0,0


In [38]:
compliant = all_data[all_data['compliance_status'] == 1]
print(compliant['hash'].nunique())
compliant.shape

7298000


(8710792, 26)

In [40]:
non_compliant = all_data[all_data['compliance_status'] == -1]
print(non_compliant['hash'].nunique())
non_compliant.shape

1010


(2630, 26)

In [41]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [42]:
compliant_gt = compliant['included_into_next_block']
compliant_pred = compliant['prediction']


print(f'Precision {precision_score(compliant_gt, compliant_pred):0.3f}')
print(f'Recall {recall_score(compliant_gt, compliant_pred):0.3f}')
print(f'F1 {f1_score(compliant_gt, compliant_pred):0.3f}')

Precision 0.943
Recall 0.895
F1 0.919


In [43]:
non_compliant_gt = non_compliant['included_into_next_block']
non_compliant_pred = non_compliant['prediction']


print(f'Precision {precision_score(non_compliant_gt, non_compliant_pred):0.3f}')
print(f'Recall {recall_score(non_compliant_gt, non_compliant_pred):0.3f}')
print(f'F1 {f1_score(non_compliant_gt, non_compliant_pred):0.3f}')

Precision 0.338
Recall 0.710
F1 0.458
